In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 

False

In [11]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█████████████████████| Rows 2277600/2277600 | Elapsed Time: 07:28 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1211551/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [ ]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

In [12]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 26.1 MB


In [5]:
import os

# Option 1: Hardcode for quick testing
os.environ["HOPSWORKS_API_KEY"] = "NED5z1BDIN3LWVy1.zo5ROYxBl57FgFYTnVyXlFa26OUGFF4lpeaIgvbeiwiCG3Ow1oVUEmgATUXLOcGE"

# Option 2 (preferred): Store your key in .env and load it
# from dotenv import load_dotenv
# load_dotenv()
# # Then in .env: HOPSWORKS_API_KEY=YOUR_KEY


In [7]:
import hopsworks

project = hopsworks.login(
    project="NYC_taxi_data_pro",            # Replace with your project name
    api_key_value=os.environ["HOPSWORKS_API_KEY"]
)
print("Logged into Hopsworks project:", project.name)


2025-03-02 20:08:35,321 INFO: Initializing external client
2025-03-02 20:08:35,322 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 20:08:39,568 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215646
Logged into Hopsworks project: NYC_taxi_data_pro


In [9]:
fs = project.get_feature_store()


In [11]:
feature_group = fs.create_feature_group(
    name="taxi_nyc",       # or any other name
    version=1,            # bump version if re-creating
    description="NYC Taxi data feature group",
    primary_key=["pickup_hour", "pickup_location_id"],  # your unique columns
    online_enabled=True    # if you want real-time serving
)


In [21]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [35]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2023
# to_year = datetime.now().year
to_year = 2024
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2023 to 2024
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading dat

In [37]:
rides

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107
...,...,...
37463263,2023-12-31 23:04:34,233
37463264,2023-12-31 23:08:15,48
37463265,2023-12-31 23:16:15,196
37463266,2023-12-31 23:21:58,140


In [41]:
rides.shape

(37463268, 2)

In [43]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [45]:
ts_data.shape

(2277600, 3)

In [47]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277600 entries, 0 to 2277599
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 26.1 MB


In [49]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
2277595,2023-12-31 19:00:00,263,188
2277596,2023-12-31 20:00:00,263,198
2277597,2023-12-31 21:00:00,263,232
2277598,2023-12-31 22:00:00,263,160


In [53]:
feature_store = project.get_feature_store()

In [63]:
FEATURE_GROUP_NAME = "taxi_nyc"
FEATURE_GROUP_VERSION = 1
feature_group = feature_store.get_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
)


In [69]:
print(ts_data.columns)


Index(['pickup_hour', 'pickup_location_id', 'rides'], dtype='object')


In [75]:
# Print each feature's name and data type from the feature group
for feature in feature_group.features:
    print(f"Feature name: {feature.name}, Data type: {feature.dtype}")
